In [2]:
from cvxopt import matrix, solvers
import numpy as np
import scipy
from scipy.special import comb
from itertools import combinations
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib import pyplot as plt

solvers.options['show_progress'] = False
rng = np.random.RandomState(8)

In [3]:
N = 12
M = 6
S = 5
T = 100
r_max = 30
l = 4
p_min = M/4
reward_constant = 10
sensor_constant = 2
cost_constant = 0.005
risk_constant = 0.005
cost_risk_constant = 0.05

gain = 10
delta = 0.1
drl = np.power(10, delta*(-gain/2))

print('number of attacker strategies: ', comb(S, l, exact=True)*r_max)
print(p_min)

number of attacker strategies:  150
1.5


In [4]:
def s(a):
    return np.sqrt(a)

dist_T_R = [[0, 0, 10, 20, 0, 0, 10, 20, 10, 10, 20, 20],
            [10, 0, 0, 10, 10, 0, 0, 10, s(2)*10, 10, s(5)*10, 20],
            [s(5)*10, s(2)*10, 10 ,10, 20, 10, 0, 0, 20, 10, s(5), s(2)*10],
            [s(2)*10, 10, 10, s(2)*10, 10, 0, 0, 10, 10, 0, s(2)*10, 10],
            [10, s(2)*10, s(5)*10, s(10)*10, 0, 10, 20, 30, 0, 10, 10, s(2)*10], 
            [s(5)*10, 20, 20, s(8)*10, s(2)*10, 10, 10, s(2)*10, 10, 0, 10, 0]]
dist_T_R = np.array(dist_T_R)

dist_T_S = [[10, 30, 10, s(2)*10, s(10)*10],
           [s(2)*10, 10, 20, 10, s(13)*10],
           [s(2)*10, 10, 30, 0, s(8)*10],
           [s(5)*10, s(5)*10, 20, 0, s(2)*2],
           [20, s(17)*10, 0, 10, 20],
           [s(10)*10, s(8)*10, s(5)*10, 0, s(5)*10]]
dist_T_S = np.array(dist_T_S)

walls_T_R = [[0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 3, 3],
            [2, 1, 0, 1, 2, 1, 1, 2, 3, 3, 4, 4],
            [4, 3, 2, 1, 3, 2, 1, 0, 4, 3, 5, 4],
            [1, 1, 2, 3, 1, 0, 1, 2, 2, 1, 3, 2],
            [2, 3, 4, 5, 1, 2, 3, 4, 0, 1, 1, 1],
            [3, 3, 3, 4, 2, 2, 2, 3, 2, 1, 1, 0]]
walls_T_R = np.array(walls_T_R)

walls_T_S = [[1, 4, 2, 3, 4],
            [3, 2, 3, 1, 4],
            [5, 2, 4, 2, 5],
            [2, 3, 2, 1, 3],
            [3, 5, 2, 2, 2],
            [4, 5, 4, 2, 1]]
walls_T_S = np.array(walls_T_S)

# directions 
dr = np.zeros((M, S, N))

dr[0,0,1], dr[0,0,4] = drl, drl
dr[0,1,4], dr[0,1,5], dr[0,1,6], dr[0,1,7] = drl, drl, drl, drl
dr[0,2,0], dr[0,2,3] = drl, drl
dr[0,3,1], dr[0,3,4] = drl, drl
dr[0,4,0], dr[0,4,5] = drl, drl

dr[1,0,2], dr[1,0,5] = drl, drl
dr[1,1,1], dr[1,1,6] = drl, drl
dr[1,2,1], dr[1,2,6] = drl, drl
dr[1,3,5], dr[1,3,7] = drl, drl
dr[1,4,1], dr[1,4,6] = drl, drl

dr[2,0,3], dr[2,0,6], dr[2,0,7] = drl, drl, drl
dr[2,1,6] = drl
dr[2,2,3] = drl
dr[2,3,3] = drl
dr[2,4,6], dr[2,4,7] = drl, drl

dr[3,0,6], dr[3,0,9] = drl, drl
dr[3,1,5] = drl
dr[3,2,6] = drl
dr[3,3,6] = drl
dr[3,4,5] = drl

dr[4,0,9] = drl
dr[4,1,4], dr[4,1,11] = drl, drl
dr[4,2,5] = drl
dr[4,3,4], dr[4,3,10] = drl, drl
dr[4,4,9] = drl

dr[5,0,10] = drl
dr[5,1,8] = drl
dr[5,2,10] = drl
dr[5,3,8], dr[5,3,9] = drl, drl
dr[5,4,8] = drl

###

dr[0,0,0] = 1
dr[0,1,1], dr[0,1,2], dr[0,1,3] = 1, 1, 1
dr[0,2,4] = 1
dr[0,3,5], dr[0,3,6] = 1, 1
dr[0,4,4], dr[0,4,8], dr[0,4,10] = 1, 1, 1

dr[1,0,0], dr[1,0,1] = 1, 1
dr[1,1,2], dr[1,1, 3] = 1, 1
dr[1,2,4], dr[1,2,5] = 1, 1
dr[1,3,6] = 1
dr[1,4,5], dr[1,4,8], dr[1,4,9], dr[1,4,10] = 1, 1, 1, 1

dr[2,0,0], dr[2,0,1], dr[2,0,6] = 1, 1, 1
dr[2,1,3], dr[2,1,7] = 1, 1
dr[2,2,4], dr[2,2,5], dr[2,2,6] = 1, 1, 1
dr[2,4,9], dr[2,4,10], dr[2,4,11] = 1, 1, 1

dr[3,0,0], dr[3,0,1] = 1, 1
dr[3,1,2], dr[3,1,3], dr[3,1,6], dr[3,1,7] = 1, 1, 1, 1
dr[3,2,4], dr[3,2,5] = 1, 1
dr[3,4,8], dr[3,4,9], dr[3,4,10] = 1, 1, 1

dr[4,0,0], dr[4,0,4] = 1, 1
dr[4,1,3], dr[4,1,5], dr[4,1,6] = 1, 1, 1
dr[4,3,9] = 1
dr[4,4,8], dr[4,4,10] = 1, 1

dr[5,0,0], dr[5,0,1], dr[5,0,4], dr[5,0,5], dr[5,0,9] = 1, 1, 1, 1, 1
dr[5,1,3], dr[5,1,6], dr[5,1,7] = 1, 1, 1
dr[5,2,4], dr[5,2,8], dr[5,2,9] = 1, 1, 1
dr[5,4,10], dr[5,4,11] = 1, 1

dr = np.swapaxes(dr, 1, 2)
print(dr.shape)

# mm frequency
def mm_rss(d,w):
    if d==0:
        d=1
    return -51 - 20 * np.log10(d) - w*6

def compute_T_rss(dist_T, walls):
    a, b = dist_T.shape[0], dist_T.shape[1]
    res = np.zeros((a, b))
    for i in range(a):
        for j in range(b):
            res[i,j] = mm_rss(dist_T[i,j], walls[i,j])
    return res

TR_rss = compute_T_rss(dist_T_R, walls_T_R)
TS_rss = compute_T_rss(dist_T_S, walls_T_S)

tot_rss = np.zeros((M, N, S))
for a in range(M):
    for b in range(N):
        for c in range(S):
            tot_rss[a,b,c] = TS_rss[a,c]

# tot_rss = np.multiply(tot_rss, dr)
max_tot_rss = np.max(tot_rss)
rel_tot_rss = tot_rss - max_tot_rss
rel_tot_rss = rel_tot_rss/10
CF = np.power(10, rel_tot_rss)
CF = np.multiply(CF, dr)

(6, 12, 5)


In [5]:
r_max_s = rng.randint(1, r_max+1, S)
RS = np.ones((r_max, S))
risk_ratios = rng.rand(r_max, S)
costs = np.zeros((r_max, S))
risks = np.zeros((r_max, S))
for r in range(r_max):
    for s in range(S):
        RS[r, s] = sensor_constant*np.minimum((r+1)/r_max_s[s], 1)
        costs[r, s] = cost_constant*(r+1)
        risks[r, s] = risk_constant*(risk_ratios[r, s])
        
AD = np.zeros((M, N, r_max, S))
for m in range(M):
    for n in range(N):
        for r in range(r_max):
            for s in range(S):
                AD[m, n, r, s] = CF[m, n, s]*RS[r, s]

PM = rng.rand(N, T)*reward_constant

# neighbors
nbr = [[1, 1, 0, 1, 1, 0],
      [1, 1, 0, 1, 0, 0],
      [1, 1, 1, 1, 0, 0],
      [0, 1, 1, 1, 0, 0],
      [1, 1, 0, 1, 1, 0],
      [1, 1, 1, 1, 1, 1],
      [1, 1, 1, 1, 0, 0],
      [0, 1, 1, 1, 0, 0],
      [1, 0, 0, 1, 1, 1],
      [1, 0, 1, 1, 1, 1],
      [0, 0, 0, 1, 1, 1],
      [0, 0, 0, 1, 1, 1]]

nbr = np.array(nbr)

In [6]:
# create constraint matrices per time slot

# regular constraints
G = []
h = []
# c_j,t < 1
for i in range(1, M+1, 1):
    g = np.zeros(M+1)
    g[i] = 1
    G += [g]
    h += [1]
# c_j,t > 0
for i in range(1, M+1, 1):
    g = np.zeros(M+1)
    g[i] = -1
    G += [g]
    h += [0]

G = np.array(G).astype(float)
h = np.array(h).astype(float)

# for individual time slots
full_nbr = np.hstack((np.zeros(N).reshape(N, 1), nbr)).astype(float)

p_mins = np.ones(N)*p_min
p_mins = np.sum(full_nbr, axis=1) - p_mins
G = np.vstack((G, full_nbr))
h = np.concatenate((h, p_mins))

c = np.zeros(M+1).astype(float)
c[0] = 1

print(G.shape, h.shape, c.shape)

(24, 7) (24,) (7,)


In [7]:
def compute_opt_sol_for_behavior(S, l, r_max, AD, PM, costs, risks, G, h, c, t):
    # utility constraints
    A = []
    b = []
    comb = combinations(list(np.arange(S, dtype=int)), l)
    for cm in list(comb):
        for r in range(r_max):
            list_c = list(cm)
            temp = np.dot(np.sum(AD[:, :, r, list_c], axis=2), PM[:, t])
            if (np.sum(costs[r, list_c]) + np.sum(risks[r, list_c]))/l <= cost_risk_constant:
#               print((np.sum(costs[r, list_c]) + np.sum(risks[r, list_c]))/l)
                A += [temp]
                b += [np.sum(temp)]
    A = np.array(A)
    b = np.array(b)
    A = np.hstack((np.ones(A.shape[0]).reshape((A.shape[0], 1)), A)).astype(float)
#     print(A[0])
    
    # combine constraints 
    A = np.vstack((G, -1*A))
    b = np.concatenate((h, -1*b))

    A_ = matrix(A)
    b_ = matrix(b)
    c_ = matrix(c)
    
    sol = solvers.lp(c_, A_, b_, solver='glpk')
#     sol = solvers.lp(c_, A_, b_)
    
    solution = np.array(list(sol['x']))
    status = sol['status']
    return status, solution[0], solution[1:]

In [8]:
def compute_random_C(G, h, M):
    c = rng.rand(M+1)
    c[0] = 0
#     rnd_indices = rng.choice(np.arange(1,M+1), int(M/2), replace=False)
#     c[rnd_indices] = 0
    A_ = matrix(G)
    b_ = matrix(h)
    c_ = matrix(-1*c)
    
    sol = solvers.lp(c_, A_, b_, solver='glpk')
#     sol = solvers.lp(c_, A_, b_)    
    solution = np.array(list(sol['x']))
    return solution

In [9]:
def compute_max_util_for_C(C, S, l, r_max, AD, PM, costs, risks, t):
    max_comb = None
    max_util = float('-inf')
    comb = combinations(list(np.arange(S, dtype=int)), l)
    for cm in list(comb):
        for r in range(r_max):
            list_c = list(cm)
            temp = np.dot(np.sum(AD[:, :, r, list_c], axis=2), PM[:, t])
            if (np.sum(costs[r, list_c]) + np.sum(risks[r, list_c]))/l <= cost_risk_constant:
                util = np.sum(temp)
            else:
                util = float('-inf')
            if util > max_util:
                max_util = util
                max_comb = list_c
    return max_util, max_comb

In [10]:
opts = []
max_utils = []
for t in range(T):
    status, opt, sol = compute_opt_sol_for_behavior(S, l, r_max, AD, PM, costs, risks, G, h, c, t)
    C = sol
    feasible_sol = compute_random_C(G, h, M)[1:]
    max_util, max_comb = compute_max_util_for_C(feasible_sol, S, l, r_max, AD, PM, costs, risks, t)
    opts +=[opt]
    max_utils += [max_util]

In [11]:
print('average opt: ', np.mean(opts))
print('average rand max payoff: ', np.mean(max_utils))

average opt:  1.6158494596856827
average rand max payoff:  6.126663343799598


In [12]:
print

<function print>